# Vibe Coding: Real-World Data Cleaning Challenge

## The Mission

You're a Data Analyst at **TechSalary Insights**. Your manager needs answers to critical business questions, but the data is messy. Your job is to clean it and provide accurate insights.

**The catch:** You must figure out how to clean the data yourself. No step by step hints just you, your AI assistant, and real world messy data.

---

## The Dataset: Ask A Manager Salary Survey 2021

**Location:** `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`

This is **real survey data** from Ask A Manager's 2021 salary survey with over 28,000 responses from working professionals. The data comes from this survey: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

**Why this dataset is perfect for vibe coding:**
- Real human responses (inconsistent formatting)
- Multiple currencies and formats  
- Messy job titles and location data
- Missing and invalid entries
- Requires business judgment calls

---

## Your Business Questions

Answer these **exact questions** with clean data. There's only one correct answer for each:

### Core Questions (Required):
1. **What is the median salary for Software Engineers in the United States?** 
2. **Which US state has the highest average salary for tech workers?**
3. **How much does salary increase on average for each year of experience in tech?**
4. **Which industry (besides tech) has the highest median salary?**

### Bonus Questions (If time permits):
5. **What's the salary gap between men and women in tech roles?**
6. **Do people with Master's degrees earn significantly more than those with Bachelor's degrees?**

**Success Criteria:** Your final answers will be compared against the "official" results. Data cleaning approaches can vary, but final numbers should be within 5% of expected values.


---
# Your Work Starts Here

## Step 0: Create Your Plan
**Before writing any code, use Cursor to create your todo plan. Then paste it here:**

## My Data Cleaning Plan

*(Paste your Cursor todo list here)*

- [ ] Example todo item
- [ ] Another example
- [ ] ...


## Step 1: Data Loading and Exploration

Start here! Load the dataset and get familiar with what you're working with.


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Step 2: Data Cleaning


In [2]:
df = pd.read_csv('/workspaces/-MSK--ds-fall-2025-wed/Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv', sep = '\t')

columns = ['Job title', 'If your job title needs additional context, please clarify here:', 
           'If your income needs additional context, please provide it here:', 
           'What country do you work in?', "If your income needs additional context, please provide it here:", 'What city do you work in?']

for column in columns:
    df[column] = df[column].str.title()



## Step 3: Business Questions Analysis

Now answer those important business questions!


In [3]:
# Question 1: What is the median salary for Software Engineers in the United States?

# Initial Cleaning
US_df = df[(df["If you're in the U.S., what state do you work in?"].notnull()) & (df['Please indicate the currency'] == 'USD')]

US_df = US_df[US_df['What is your annual salary? (You\'ll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)'].notnull()]

US_df['How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.'] = US_df['How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.'].fillna(0)

US_df['What is your annual salary? (You\'ll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)'] = US_df['What is your annual salary? (You\'ll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)'].str.replace(r'[^\d.]', '', regex=True)

base = US_df['What is your annual salary? (You\'ll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)'].astype(float)
additional = US_df['How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.']

US_df['total_salary'] = base + additional

# Outliers
Q1 = US_df['total_salary'].quantile(0.25)
Q3 = US_df['total_salary'].quantile(0.75)
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

US_df = US_df[
    (US_df['total_salary'] <= upper_bound) & 
    (US_df['total_salary'] >= lower_bound)
]

se_usdf = US_df[US_df['Job title'] == 'Software Engineer']

print(se_usdf['total_salary'].median())

123000.0


In [4]:
# Question 2: Which US state has the highest average salary for tech workers?

state_df = US_df.copy()

state_df['If you\'re in the U.S., what state do you work in?'] = state_df['If you\'re in the U.S., what state do you work in?'].str.title()
state_df['If you\'re in the U.S., what state do you work in?'] = state_df['If you\'re in the U.S., what state do you work in?'].str.replace(' ','')
state_df['If you\'re in the U.S., what state do you work in?'] = state_df['If you\'re in the U.S., what state do you work in?'].str.replace('DistrictOfColumbia','Washington')

state_df['final_states'] = state_df['If you\'re in the U.S., what state do you work in?'].str.split(',')
state_df = state_df.explode('final_states')

mean = state_df.groupby('final_states')['total_salary'].mean()
mean.sort_values(ascending = False)

final_states
California       105358.861279
Washington        97679.012935
Massachusetts     95010.784669
NewYork           94876.450405
Virginia          90073.260471
Maryland          88980.626998
Oregon            88510.001645
Illinois          88135.749144
NewJersey         87283.912371
Colorado          86967.317881
Texas             86702.739454
Delaware          86416.234043
Connecticut       85176.162281
Nevada            84968.968750
Minnesota         84158.502104
Georgia           84105.725528
Utah              82483.064677
Arizona           82028.684385
Pennsylvania      81709.181522
NorthCarolina     80254.661538
NewMexico         79911.958333
RhodeIsland       79008.947368
Florida           78250.475248
Alaska            77046.461538
Michigan          76674.003724
Missouri          76324.781711
NewHampshire      75748.933884
Wisconsin         75639.648590
Ohio              75561.890432
Vermont           74553.611111
Louisiana         73803.976744
Indiana           73278.47

In [15]:
# Question 3: How much does salary increase on average for each year of experience in tech?
job_title = 'Software|Data|IT|Programmer|Developer|Cloud|Quant|Network|Cyber|Web'
tech = US_df[(US_df['Job title'].str.contains(job_title)) | (US_df['What industry do you work in?'] == 'Computing or Tech')]
tech


mean_tech = tech.groupby('How many years of professional work experience do you have in your field?')['total_salary'].mean()
mean_tech = mean_tech.sort_values()
mean_tech

inc = {
    1:87754.785366,
    3:101808.588110,
    6:112842.438776,
    9:124367.460265,
    35:130489.068182,
    41:132137.500000,
    15:132665.513369,
    25:136108.207547
}

i = 0
j = 1
nums = [1,3,6,9,35,41,15,25]
change = []
while j < len(nums):
    a = nums[i]
    b = nums[j]
    change.append((inc[b] - inc[a])/(b-a))
    i += 1
    j +=1 
print(sum(change)/len(change))


2197.2388185208792


In [ ]:

# Question 4: What percentage of respondents work remotely vs. in-office?
remote_words = 'Remote|Online|Wfh|Telecommute|Home|Virtual|Telework|Anywhere|House'


remote_columns = [
    'If your job title needs additional context, please clarify here:',
    'If your income needs additional context, please provide it here:',
    'What country do you work in?',
    'Job title',
    'What city do you work in?'
]


# Create a mask that's True if ANY column contains remote words
s = 0
for column in remote_columns:
    s += len(df[df[column].str.contains(remote_words, na = False)])
s += df['What city do you work in?'].isna().sum()
s += df['What country do you work in?'].isna().sum()


print(f"Total remote jobs found: {s}")
print(f"Total nonremote jobs found: {len(df[df['Job title'].notnull()]) - s}")

Total remote jobs found: 814
Total nonremote jobs found: 27247


In [ ]:
# Question 5: Which industry (besides tech) has the highest median salary?
US_df['What industry do you work in?'] = US_df['What industry do you work in?'].str.title()
US_df = US_df[US_df['What industry do you work in?'].notnull()]

industry_salary = US_df.groupby('What industry do you work in?')['total_salary'].median()
industry_salary = industry_salary.sort_values(ascending = False)
industry_salary

What industry do you work in?
Pharma R&D                                                210000.0
Pharma Research                                           210000.0
Wholesale - Apparel                                       205000.0
R&D                                                       200000.0
Clean Energy (Eg. Energy Efficiency, Renewables, Etc.)    200000.0
                                                            ...   
Museum (University Affiliated)                                40.0
I'M Currently A Student And Don'T Have A Job                   0.0
Homemaker                                                      0.0
Stay-At-Home Parent                                            0.0
Student                                                        0.0
Name: total_salary, Length: 913, dtype: float64

In [17]:
# Bonus Questions:
# Question 6: What's the salary gap between men and women in similar roles?
# Question 7: Do people with Master's degrees earn significantly more than those with Bachelor's degrees?
# Question 8: Which company size (startup, medium, large) pays the most on average?
df

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research And Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,Us,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,Usa,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,Us,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28057,7/12/2024 22:52:01,35-44,Health care,Veterinarian,NaN,135000,NaN,USD,NaN,NaN,United States,Missouri,Wentzville,11 - 20 years,11 - 20 years,"Professional degree (MD, JD, etc.)",Woman,White
28058,7/23/2024 17:51:03,25-34,Computing or Tech,Systems Architect,NaN,109000,NaN,USD,NaN,NaN,Usa,Georgia,Atlanta,5-7 years,5-7 years,College degree,Man,White
28059,7/24/2024 12:22:58,18-24,"Accounting, Banking & Finance",Risk Management Associate,NaN,1200,0.0,USD,NaN,NaN,Myanmar,Colorado,Yangon,2 - 4 years,2 - 4 years,Some college,Man,Asian or Asian American
28060,7/26/2024 11:20:45,18-24,Computing or Tech,It,NaN,1700,10.0,USD,NaN,NaN,Burma,NaN,Yangon,2 - 4 years,1 year or less,Some college,Man,Asian or Asian American


## Final Summary

**Summarize your findings here:**

1. **Median salary for Software Engineers in US:** $123,000
2. **Highest paying US state for tech:** California
3. **Salary increase per year of experience:** $2197.24 per year
4. **Remote vs office percentage:** 2.5% remote, 97.5% office (For this question, I define office as anything not fully remote)(I could tell it should be closer to ~25%)
5. **Highest paying non-tech industry:** Pharma R&D

**Key insights:**
- Data is really messy.
- Data handling really is a developed skill.
- Generating insights is tedious. 

**Challenges faced:**
- String data was super unorganized. Had to title it then search through using regex or filtering
- Forgot about outliers. I decided to just drop then using the IQR.

**What you learned about vibe coding:**
- AI really good at giving suggestions
- You need to be able to read code to debug
- You can't really solely rely on vibe coding
